# Text Summary & Scoring Project
##### Michael Creegan, Yungfeng Dai, Hong Gyu Ji, Ziling Zeng
##### Python for Data Analysis
##### Columbia University

# Abstract

Summarization is a common problem as we all have mountains of data in datalakes and on the web. Summarization of data can be very useful to see whether or not something is what you are loooking for or whether it's worth reading. Another use case could could be to store summaries of articles and store it in the backend and run downstream tasks on that data. It could also be useful to understand whether or not that summary is "good" or not that indicates the quality of the summary. 

To explore this topic, we will leverage the extreme summarization dataset from which consists of BBC articles accompanying single sentence summaries. Each article is prefaced with an introductory sentence (which is a summary) that is professionally written, typically by the author of the article.

To summarize articles, we will use an encoder-decoder transformer (sequence-to-sequence) which combines  decodes and encoders because we need to perform both input and output tasks (taking in text and then generating a summary). We selected this type of transformer because the encoder accepts inputs (text) and computes a high level representation of those inputs  which are then passed to the decoder to generate a prediction output (summary). This has advantages over using a standalone encoder like BERT/ALBERT/ELECTRA/RoBERTA/DistilBERT to name a few because  encoders are pre-trained by filling randomly masked words in sentences and therefore are better suited for output tasks. Using a standalone decoder like gpt2 would also not be optimal because decoders are trained to guess the next word in a sequence (left or right context aka does not have context on one side of the sequence) and therefore are better suited at generating text but not necessarily taking in text because of the hidden context limitations. 

Our scoring will compare the output of the BART encoder-decoder model to the professionally written summaries in the XSUM dataset to see how similar a machine generated summary is to a professional one.

# Importing Transformers & Dependencies

In [1]:
import pandas as pd
import numpy as np
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig
from datasets import load_dataset, load_metric
import random
from IPython.display import display, HTML

# Load XSUM Dataset

In [2]:
xsum = load_dataset('xsum')

Using custom data configuration default
Reusing dataset xsum (C:\Users\creeg\.cache\huggingface\datasets\xsum\default\1.2.0\32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934)
100%|██████████| 3/3 [00:00<00:00, 52.67it/s]


### We can see that the dataset is a "DatasetDict" where the keys are strings that correspond to the split and the values are the dataset object. In the XSUM dataset, the the keys are "training", "validation", and "test" with values corresponding to "document", "summary", and "id" (columns)

In [3]:
xsum

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

### View a record of the underlying data

In [4]:
xsum['test'][0]

{'document': 'Prison Link Cymru had 1,099 referrals in 2015-16 and said some ex-offenders were living rough for up to a year before finding suitable accommodation.\nWorkers at the charity claim investment in housing would be cheaper than jailing homeless repeat offenders.\nThe Welsh Government said more people than ever were getting help to address housing problems.\nChanges to the Housing Act in Wales, introduced in 2015, removed the right for prison leavers to be given priority for accommodation.\nPrison Link Cymru, which helps people find accommodation after their release, said things were generally good for women because issues such as children or domestic violence were now considered.\nHowever, the same could not be said for men, the charity said, because issues which often affect them, such as post traumatic stress disorder or drug dependency, were often viewed as less of a priority.\nAndrew Stevens, who works in Welsh prisons trying to secure housing for prison leavers, said the

### We can use a function to view a random selection of articles and summaries in the training section (largest section) to get a more accurate depiction of what the data looks like in an easier to read format

In [5]:
def display_function(xsum, num_examples=5):
    assert num_examples <= len(xsum)                # limit to number of records in the xsum
    
    selections = []                                      # create empty list to put the records into 
    
    for _ in range(num_examples):                   # we can use _ here in place of a variable name because we don't care how many time sthe loop is run
        selection = random.randint(0, len(xsum) - 1)
        while selection in selections:
            selection = random.randint(0, len(xsum) - 1)
        selections.append(selection)

    xsumPd = pd.DataFrame(xsum[selections])
    for column, typ in xsum.features.items():
        display(HTML(xsumPd.to_html()))

### Our end goal is to create accurate summaries using this model so we need to remove the text characters that do not provide any contextual value. We can also see that there are characters in the document that are not present in the summary which could cause discrepencies between our machine generated summary vs the professional human generated one. We need to remove new line characters that are present in the document column but not the summary column

In [6]:
display_function(xsum["test"])

### We can address the problem we mentioned above by define a cleaning function that replaces new lines with white space.

In [7]:
def clean(row):
    row['document'] = row['document'].replace('\n', ' ')
    return row

### We can now apply the cleaning function we created and map it onto our data (it loads for train, test, and validation)

In [8]:
xsum = xsum.map(clean)

Loading cached processed dataset at C:\Users\creeg\.cache\huggingface\datasets\xsum\default\1.2.0\32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934\cache-ec5b3ab440c9df82.arrow
Loading cached processed dataset at C:\Users\creeg\.cache\huggingface\datasets\xsum\default\1.2.0\32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934\cache-a176a692461cda61.arrow
Loading cached processed dataset at C:\Users\creeg\.cache\huggingface\datasets\xsum\default\1.2.0\32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934\cache-bc530be4c3ab51ba.arrow


### Voila!

In [9]:
display_function(xsum["test"])

,document,summary,id
0,He was taken to the Royal Sussex County Hospital following the incident just before 06:00 BST on Saturday. He is reported to be in a stable condition. Sussex Police said the incident in Preston Street appeared to be linked to the collapse of a section of railing surrounding the balcony. The one-way street had to be closed to allow for investigations.,A 20-year old man is in hospital with serious injuries after he fell from a first floor balcony in Brighton.,33952455
1,"Research carried out by the Fostering Network suggests almost half of fostered young people are already living with their third foster family since going into care. The group has warned that 750 more foster carers are ""urgently"" needed to meet the demands of the care system. It urged people to ""open their hearts and homes"" to vulnerable youngsters. Currently, more than 5,500 children are in foster care in Scotland, living with 4,400 families and carers. The Fostering Network surveyed 250 children, teenagers and foster carers across Scotland and discovered that many young people had failed to find stability. Almost half were already living with their third family, a quarter were with their fourth family and about 20 were living with their 10th family since going into care. There was a particular need for homes to be found for vulnerable teenagers, siblings and disabled children, the study found. Carla, 23, was taken into care at the age of 12 and had eight foster homes before moving in with the Randalls. ""Looking back now I realised that the Randalls saved my life,"" she said. ""I never understood the extent of the neglect and abuse I had endured until I came to live with a 'normal' loving family. ""They were just always themselves, the smallest details meant so much to me. ""They nurtured a young, angry, untrusting teenager to become a positive, empathetic and successful young woman."" The Fostering Network said instability had a detrimental effect on the child's education and wellbeing, while finding a stable foster carer from the outset could lead to improved relationships and a happier childhood. Sara Lurie, director of the Fostering Network Scotland, said: ""As each year passes, we see more and more children coming into care. ""We need people who can open their heart, and their homes, to vulnerable children and young people and use their skills to help support them to reach their full potential. ""In particular we need people who have the skills, patience and passion to look after teenagers who may have had a really tough time and be facing some real challenges, and to offer them love, stability and security. ""A good foster carer will believe in the ambition of the children in their care in the same way they'd believe in the ambition of their biological family members."" Apologies for the delay, see below as requested. A Scottish government spokeswoman said: ""Giving young people security is paramount and we have done a great deal of work with our partners across local government and the third sector to improve how we intervene early when there is a problem within families to find appropriate solutions quickly. ""We have also expanded the age at which young people can remain in foster care as part of the continuing care provisions and the support available when they transition into independent living.""","Teenagers in foster care in Scotland are being moved too often, according to a campaign group.",32952046
2,"The advert for its invisible deodorant range said ""white is purity"" across an image of a woman. It was posted on the firm's Facebook page, geographically aimed at its followers in the Middle East. Nivea accepted the post was ""misleading"". Many on social media complained the post was racist, but it was also widely-shared among alt-right users. The advert on the Nivea Facebook page (since deleted) was posted alongside the caption: ""Keep it clean, keep it bright. Don't let anything ruin it."" The post was directed at its

,document,summary,id
0,He was taken to the Royal Sussex County Hospital following the incident just before 06:00 BST on Saturday. He is reported to be in a stable condition. Sussex Police said the incident in Preston Street appeared to be linked to the collapse of a section of railing surrounding the balcony. The one-way street had to be closed to allow for investigations.,A 20-year old man is in hospital with serious injuries after he fell from a first floor balcony in Brighton.,33952455
1,"Research carried out by the Fostering Network suggests almost half of fostered young people are already living with their third foster family since going into care. The group has warned that 750 more foster carers are ""urgently"" needed to meet the demands of the care system. It urged people to ""open their hearts and homes"" to vulnerable youngsters. Currently, more than 5,500 children are in foster care in Scotland, living with 4,400 families and carers. The Fostering Network surveyed 250 children, teenagers and foster carers across Scotland and discovered that many young people had failed to find stability. Almost half were already living with their third family, a quarter were with their fourth family and about 20 were living with their 10th family since going into care. There was a particular need for homes to be found for vulnerable teenagers, siblings and disabled children, the study found. Carla, 23, was taken into care at the age of 12 and had eight foster homes before moving in with the Randalls. ""Looking back now I realised that the Randalls saved my life,"" she said. ""I never understood the extent of the neglect and abuse I had endured until I came to live with a 'normal' loving family. ""They were just always themselves, the smallest details meant so much to me. ""They nurtured a young, angry, untrusting teenager to become a positive, empathetic and successful young woman."" The Fostering Network said instability had a detrimental effect on the child's education and wellbeing, while finding a stable foster carer from the outset could lead to improved relationships and a happier childhood. Sara Lurie, director of the Fostering Network Scotland, said: ""As each year passes, we see more and more children coming into care. ""We need people who can open their heart, and their homes, to vulnerable children and young people and use their skills to help support them to reach their full potential. ""In particular we need people who have the skills, patience and passion to look after teenagers who may have had a really tough time and be facing some real challenges, and to offer them love, stability and security. ""A good foster carer will believe in the ambition of the children in their care in the same way they'd believe in the ambition of their biological family members."" Apologies for the delay, see below as requested. A Scottish government spokeswoman said: ""Giving young people security is paramount and we have done a great deal of work with our partners across local government and the third sector to improve how we intervene early when there is a problem within families to find appropriate solutions quickly. ""We have also expanded the age at which young people can remain in foster care as part of the continuing care provisions and the support available when they transition into independent living.""","Teenagers in foster care in Scotland are being moved too often, according to a campaign group.",32952046
2,"The advert for its invisible deodorant range said ""white is purity"" across an image of a woman. It was posted on the firm's Facebook page, geographically aimed at its followers in the Middle East. Nivea accepted the post was ""misleading"". Many on social media complained the post was racist, but it was also widely-shared among alt-right users. The advert on the Nivea Facebook page (since deleted) was posted alongside the caption: ""Keep it clean, keep it bright. Don't let anything ruin it."" The post was directed at its

,document,summary,id
0,He was taken to the Royal Sussex County Hospital following the incident just before 06:00 BST on Saturday. He is reported to be in a stable condition. Sussex Police said the incident in Preston Street appeared to be linked to the collapse of a section of railing surrounding the balcony. The one-way street had to be closed to allow for investigations.,A 20-year old man is in hospital with serious injuries after he fell from a first floor balcony in Brighton.,33952455
1,"Research carried out by the Fostering Network suggests almost half of fostered young people are already living with their third foster family since going into care. The group has warned that 750 more foster carers are ""urgently"" needed to meet the demands of the care system. It urged people to ""open their hearts and homes"" to vulnerable youngsters. Currently, more than 5,500 children are in foster care in Scotland, living with 4,400 families and carers. The Fostering Network surveyed 250 children, teenagers and foster carers across Scotland and discovered that many young people had failed to find stability. Almost half were already living with their third family, a quarter were with their fourth family and about 20 were living with their 10th family since going into care. There was a particular need for homes to be found for vulnerable teenagers, siblings and disabled children, the study found. Carla, 23, was taken into care at the age of 12 and had eight foster homes before moving in with the Randalls. ""Looking back now I realised that the Randalls saved my life,"" she said. ""I never understood the extent of the neglect and abuse I had endured until I came to live with a 'normal' loving family. ""They were just always themselves, the smallest details meant so much to me. ""They nurtured a young, angry, untrusting teenager to become a positive, empathetic and successful young woman."" The Fostering Network said instability had a detrimental effect on the child's education and wellbeing, while finding a stable foster carer from the outset could lead to improved relationships and a happier childhood. Sara Lurie, director of the Fostering Network Scotland, said: ""As each year passes, we see more and more children coming into care. ""We need people who can open their heart, and their homes, to vulnerable children and young people and use their skills to help support them to reach their full potential. ""In particular we need people who have the skills, patience and passion to look after teenagers who may have had a really tough time and be facing some real challenges, and to offer them love, stability and security. ""A good foster carer will believe in the ambition of the children in their care in the same way they'd believe in the ambition of their biological family members."" Apologies for the delay, see below as requested. A Scottish government spokeswoman said: ""Giving young people security is paramount and we have done a great deal of work with our partners across local government and the third sector to improve how we intervene early when there is a problem within families to find appropriate solutions quickly. ""We have also expanded the age at which young people can remain in foster care as part of the continuing care provisions and the support available when they transition into independent living.""","Teenagers in foster care in Scotland are being moved too often, according to a campaign group.",32952046
2,"The advert for its invisible deodorant range said ""white is purity"" across an image of a woman. It was posted on the firm's Facebook page, geographically aimed at its followers in the Middle East. Nivea accepted the post was ""misleading"". Many on social media complained the post was racist, but it was also widely-shared among alt-right users. The advert on the Nivea Facebook page (since deleted) was posted alongside the caption: ""Keep it clean, keep it bright. Don't let anything ruin it."" The post was directed at its

### We can view the column names and data types without our dataset using .features

In [10]:
xsum['test'].features

{'document': Value(dtype='string', id=None),
 'summary': Value(dtype='string', id=None),
 'id': Value(dtype='string', id=None)}

In [11]:
print(xsum['test'].info)

DatasetInfo(description='\nExtreme Summarization (XSum) Dataset.\n\nThere are three features:\n  - document: Input news article.\n  - summary: One sentence summary of the article.\n  - id: BBC ID of the article.\n\n', citation="\n@article{Narayan2018DontGM,\n  title={Don't Give Me the Details, Just the Summary! Topic-Aware Convolutional Neural Networks for Extreme Summarization},\n  author={Shashi Narayan and Shay B. Cohen and Mirella Lapata},\n  journal={ArXiv},\n  year={2018},\n  volume={abs/1808.08745}\n}\n", homepage='https://github.com/EdinburghNLP/XSum/tree/master/XSum-Dataset', license='', features={'document': Value(dtype='string', id=None), 'summary': Value(dtype='string', id=None), 'id': Value(dtype='string', id=None)}, post_processed=None, supervised_keys=SupervisedKeysData(input='document', output='summary'), task_templates=None, builder_name='xsum', config_name='default', version=1.2.0, splits={'train': SplitInfo(name='train', num_bytes=479206615, num_examples=204045, data

# Preparing XSUM Data
Before we can put the text into a model need to transform the text into a format that the transformer can understand. Encoders and decoders understand numerical values not strings so we need to tokenize each of the words and then convert them into numerical values the model can understand. The tokenization transformer splits text into tokens and then adds special tokens if it expects them to classify. The tokenizer then matches each token to unique id in vocabulary of tokenizer which has a corresponding vector of numerical values. These vectors contain the value of a word but contextualized. For example, the vector representation of the word "to" isnt just "to", it also takes iinto account the words around it which are called context (right and left context). To continue this example, "Welcome to NYC" is a sentence that has the word "to". For the word "to" the left context is "Welcome" and the right context is "NYC". The output is based on these contexts; this is how the value is a contextualized vector thanks to self-attention mechanism. We can do all of this using the AutoTokenizer.from_pretarined method to ensure that we get a tokenizer that corresponds to the model architecture we want to use (facebook/bart-large-cnn); however, we will specifically reference the BartTokenizer in our checkpoint, tokenizer, and model to ensure all aspects of our model were trained using the same methodologies so we can avoid unexpected summaries

In [12]:
checkpoint = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(checkpoint)
model = BartForConditionalGeneration.from_pretrained(checkpoint)

### We now write a function that preprocesses the test data by passing it to the tokenizer. We need to use the argument truncation=True to ensure that any input longer than the model can handle will be truncated to the maximum length alowed. We can view this information in the model config. BART has a maximum length of 1024 which we can see in max_position_embeddings

In [13]:
model.config

BartConfig {
  "_name_or_path": "facebook/bart-large-cnn",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "force_bos_token_to_be_generated": true,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "L

### We can now create the function with the maximum length allowed as per the config and an arbitrary minimum length. 

In [14]:
max_input_length = 1024
max_target_length = 100


def preperation_function(examples):
    inputs = [doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding=True)

    
    with tokenizer.as_target_tokenizer(): # Setup the tokenizer for summaries where "as_target_tokenizer" is what provides passes along the context for each vector
        labels = tokenizer(
            examples["summary"], max_length=max_target_length, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

### We can apply this function to our dataset using map

In [15]:
tokenized_xsum = xsum.map(preperation_function, batched=True)

100%|██████████| 12/12 [00:19<00:00,  1.60s/ba]


In [16]:
tokenized_xsum

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'document', 'id', 'input_ids', 'labels', 'summary'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['attention_mask', 'document', 'id', 'input_ids', 'labels', 'summary'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['attention_mask', 'document', 'id', 'input_ids', 'labels', 'summary'],
        num_rows: 11334
    })
})

In [17]:
tokenized_xsum['test'].features

{'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'document': Value(dtype='string', id=None),
 'id': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'summary': Value(dtype='string', id=None)}

### The attention mask tells the model what to pay attention to by passing values of 1 for tokens to consider and values of 0 for tokens to ignore. The input ids are the numerical mapping of tokens to BART's vocabulary; each word in BART's vocabulary is assigned a numerical value.

In [18]:
display_function(tokenized_xsum['test'])

# 

In [19]:
# def bartSummary(row):
#     # has to return a dictionary where key is name of column and key is value you want in that row
#     tokenizer.convert_ids_to_tokens(tokenized_xsum["input_ids"])
#     return {"bartSummary": "hello!"}

In [20]:
tokenized_xsum['test'].features

{'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'document': Value(dtype='string', id=None),
 'id': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'summary': Value(dtype='string', id=None)}

In [21]:
article1 = tokenized_xsum['test']['input_ids'][0]
article2 = tokenized_xsum['test']['input_ids'][1]
article3 = tokenized_xsum['test']['input_ids'][2]
article4 = tokenized_xsum['test']['input_ids'][3]
article5 = tokenized_xsum['test']['input_ids'][4]
article6 = tokenized_xsum['test']['input_ids'][5]
article7 = tokenized_xsum['test']['input_ids'][7]
article8 = tokenized_xsum['test']['input_ids'][8]
article9 = tokenized_xsum['test']['input_ids'][9]
article10 = tokenized_xsum['test']['input_ids'][10]


In [22]:
import torch
input = torch.tensor(article1)
input

tensor([    0, 21077,  4060,  ...,     1,     1,     1])

In [23]:
print([tokenizer.decode(g, skip_special_tokens=True) for g in article1])

['<s>', 'Pr', 'ison', ' Link', ' C', 'ym', 'ru', ' had', ' 1', ',', '0', '99', ' referrals', ' in', ' 2015', '-', '16', ' and', ' said', ' some', ' ex', '-', 'off', 'enders', ' were', ' living', ' rough', ' for', ' up', ' to', ' a', ' year', ' before', ' finding', ' suitable', ' accommodation', '.', ' Workers', ' at', ' the', ' charity', ' claim', ' investment', ' in', ' housing', ' would', ' be', ' cheaper', ' than', ' jail', 'ing', ' homeless', ' repeat', ' offenders', '.', ' The', ' Welsh', ' Government', ' said', ' more', ' people', ' than', ' ever', ' were', ' getting', ' help', ' to', ' address', ' housing', ' problems', '.', ' Changes', ' to', ' the', ' Housing', ' Act', ' in', ' Wales', ',', ' introduced', ' in', ' 2015', ',', ' removed', ' the', ' right', ' for', ' prison', ' le', 'avers', ' to', ' be', ' given', ' priority', ' for', ' accommodation', '.', ' Prison', ' Link', ' C', 'ym', 'ru', ',', ' which', ' helps', ' people', ' find', ' accommodation', ' after', ' their', '